# Deploy the Application Package

Deploying an Application Package using the OGC API Processes API uses the API resource highlighted in the table below:


| **Resource**                   | **Path**                                  | **Purpose**                                                                     | **Part**   |
|--------------------------------|-------------------------------------------|---------------------------------------------------------------------------------|------------|
| Landing page                   | `/`                                       | Top-level resource serving as an entry point.                                   | Part 1     |
| Conformance declaration        | `/conformance`                            | Information about the functionality supported by the server.                    | Part 1     |
| API Definition                 | `/api`                                    | Metadata about the API itself.                                                  | Part 1     |
| Process list                   | `/processes`                              | Lists available processes with identifiers and links to descriptions.           | Part 1     |
| Process description            | `/processes/{processID}`                  | Retrieves detailed information about a specific process.                        | Part 1     |
| Process execution              | `/processes/{processID}/execution`        | Executes a process, creating a job.                                             | Part 1     |
| **Deploy Process**             | **`/processes` (POST)**                   | **Deploys a new process on the server.**                                        | **Part 2** |
| Replace Process                | `/processes/{processID}` (PUT)            | Replaces an existing process with a new version.                                | Part 2     |
| Undeploy Process               | `/processes/{processID}` (DELETE)         | Removes an existing process from the server.                                    | Part 2     |
| EO Application Package         | `/processes/{processID}/package` (GET)    | Get the EOAP associated with a deployed process.                                | Part 2     |
| Job status info                | `/jobs/{jobID}`                           | Retrieves the current status of a job.                                          | Part 1     |
| Job results                    | `/jobs/{jobID}/results`                   | Retrieves the results of a job.                                                 | Part 1     |
| Job list                       | `/jobs`                                   | Retrieves a list of submitted jobs.                                             | Part 1     |
| Job deletion                   | `/jobs/{jobID}`                           | Cancels and deletes a job.                                                      | Part 1     |


This resource permits the deployment of the an Application Package and provide two options for the `Content-Type`.

## Hands-on - Deploy the water_bodies Application Package

**Encoding Options**

The deployment can use two encodings that are based on the same CWL conformance class. Both methods utilize the same water_bodies.cwl file, but the way the file is provided differs:
- OGC Application Package Encoding (application/ogcapppkg+json): This method allows you to reference the CWL file by providing its location, rather than including the file's content in the request.
- CWL Encoding (application/cwl+yaml): This method requires the CWL file content to be included directly in the request body.

**Request Configuration**

When selecting a content type, the request body text area updates to contain a relevant payload template for that encoding.
Warning: If the payload is manually edited, switching to a different encoding may not refresh the text area. In this case, the Reset button can be used to restore the appropriate template.

**Server Response**

After executing the deployment request, the server responds with a process summary similar to the one obtained from the previous process listing endpoint.
The server’s response includes a Location header containing the URL to access the detailed process description.

**Next Steps** 

You can either:
- Return to the list of available processes to verify the newly deployed process.
- Proceed to the next step to review the process description in detail

## Setup

Lists available processes in the `acme` namespace.

> **_NOTE:_**: if the `acme` namespace does not exist, ZOO Project will create it.

In [ ]:
import yaml
import requests
from loguru import logger
from pystac.item_collection import ItemCollection
from ogc_api_client.api_client import Configuration
from ogc_api_client.api_client_wrapper import ApiClientWrapper

namespace = "acme"

# ogc_api_endpoint = f"http://zoo-project-dru-service/{namespace}/ogc-api"
ogc_api_endpoint = f"http://localhost:8080/{namespace}/ogc-api"
# ogc_api_endpoint = f"http://localhost:8080/ogc-api"

configuration = Configuration(
    host=ogc_api_endpoint,
)
client = ApiClientWrapper(configuration=configuration)


def list_processes():
    processes = client.get_processes()
    print("PROCESS LIST")
    for process in processes.processes:
        print("* {0} ({1})".format(process.id, process.title))

list_processes()



If the application package was deployed previously, delete it.

In [ ]:
def undeploy(process_id):
    r = requests.delete(f"{ogc_api_endpoint}/processes/{process_id}")

    return r

app_package_entry_point = "water-bodies"

r = undeploy(app_package_entry_point)

r.status_code

## OGC Application Package Encoding (application/ogcapppkg+json)

In [ ]:
headers = {"accept": "application/json",
          "Content-Type": "application/ogcapppkg+json"}

app_package_entry_point = "water-bodies"

response = requests.post(
    
    f"{ogc_api_endpoint}/processes/?w={app_package_entry_point}",
    headers=headers,
    json = {
      "executionUnit": {
        "href": "https://github.com/eoap/mastering-app-package/releases/download/1.0.0/app-water-bodies-cloud-native.1.0.0.cwl",
        "type": "application/cwl"
      }
    }
)

In [ ]:
response.json()

In [ ]:
dict(response.headers)

## CWL Encoding (application/cwl+yaml)

If the application package was deployed previously, delete it.

In [ ]:
app_package_entry_point = "water-bodies"

undeploy(app_package_entry_point)

Download the application package from https://github.com/eoap/mastering-app-package/releases/download/1.0.0/app-water-bodies-cloud-native.1.0.0.cwl

In [ ]:
r = requests.get("https://github.com/eoap/mastering-app-package/releases/download/1.0.0/app-water-bodies-cloud-native.1.0.0.cwl")

app_package_content  = yaml.safe_load(r.content)

app_package_content

In [ ]:
app_package_entry_point = "water-bodies"

headers = {"accept": "application/json", 
          "Content-Type": "application/cwl+yaml"}

response = requests.post(
    f"{ogc_api_endpoint}/processes?w={app_package_entry_point}",
    headers=headers,
    json=app_package_content
)

response.status_code

In [ ]:
response.json()

In [ ]:
dict(response.headers)

**Next Steps** 

You can either:
- Return to the list of available processes to verify the newly deployed process.
- Proceed to the next step to review the process description in detail